<a href="https://colab.research.google.com/github/walkerjian/Physics/blob/main/UnravelledBliss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import math

def generate_fft_code_python(N, filename):
    with open(filename, 'w') as file:
        # Start of the C code
        file.write("#include <complex.h>\n#include <stdio.h>\n\n")
        file.write(f"void fft_unrolled_{N}(double* x, complex double* X) {{\n")

        log2N = int(math.log2(N))

        for stage in range(1, log2N + 1):
            span = 1 << stage
            half_span = span // 2
            file.write(f"    // Stage {stage}\n")

            for k in range(0, N, span):
                # First half without twiddles
                file.write(f"    {{\n        double t = x[{k}];\n")
                file.write(f"        x[{k}] = t + x[{k + half_span}];\n")
                file.write(f"        x[{k + half_span}] = t - x[{k + half_span}];\n    }}\n")

                # Second half with twiddles
                for j in range(1, half_span):
                    twiddle_real = math.cos(-2.0 * math.pi * j / N)
                    twiddle_imag = math.sin(-2.0 * math.pi * j / N)
                    file.write(f"    {{\n        complex double t = x[{k + j + half_span}] * ({twiddle_real} - I * {twiddle_imag});\n")
                    file.write(f"        x[{k + j + half_span}] = x[{k + j}] - t;\n")
                    file.write(f"        x[{k + j}] += t;\n    }}\n")

            file.write("\n")

        # End of the FFT logic
        file.write("    // Copy results to X\n")
        for k in range(N):
            file.write(f"    X[{k}] = x[{k}];\n")

        # Add the main function for standalone execution
        file.write(f"""}}
int main() {{
    double input[{N}];
    complex double output[{N}];

    // Read input data
    for(int i = 0; i < {N}; i++) {{
        scanf("%lf", &input[i]);
    }}

    // Call FFT function
    fft_unrolled_{N}(input, output);

    // Print FFT results
    for(int i = 0; i < {N}; i++) {{
        printf("%lf + %lfi\\n", creal(output[i]), cimag(output[i]));
    }}

    return 0;
}}
""")


In [129]:
import numpy as np

def top_hat(N):
    """Generate a top-hat waveform."""
    waveform = np.zeros(N)
    waveform[N//4:3*N//4] = 1
    return waveform

def sinc(N):
    """Generate a sinc waveform."""
    x = np.linspace(-2*np.pi, 2*np.pi, N, endpoint=False)
    return np.sinc(x)

def sin(N):
    """Generate a sin waveform."""
    x = np.linspace(-2*np.pi, 2*np.pi, N, endpoint=False)
    return np.sin(x)

def cos(N):
    """Generate a cos waveform."""
    x = np.linspace(-2*np.pi, 2*np.pi, N, endpoint=False)
    return np.cos(x)

def sincos(N):
    """Generate a sincos waveform."""
    x = np.linspace(-2*np.pi, 2*np.pi, N, endpoint=False)
    return np.sin(x)*np.cos(x)

# ... (similar functions for other waveforms: sin, gaussian, delta, sawtooth)


In [134]:
def ascii_art(data, width=256, height=40):
    """Convert data to ASCII art."""
    screen = [[' ' for _ in range(width)] for _ in range(height)]
    max_val, min_val = max(data), min(data)

    for i, value in enumerate(data):
        norm_val = 1 - (value - min_val) / (max_val - min_val)  # Invert the normalization here
        y = int(norm_val * (height - 1))
        screen[y][i * width // len(data)] = '*'

    for row in screen:
        print(''.join(row))

def ascii_art2(data, width=128, height=40):
    """Convert data to ASCII art."""
    screen = [[' ' for _ in range(width)] for _ in range(height)]
    max_val, min_val = max(data), min(data)

    for i, value in enumerate(data):
        # Normalize the value between 0 and 1
        norm_val = (value - min_val) / (max_val - min_val)

        # Calculate the number of asterisks to be printed
        num_asterisks = int(norm_val * height)

        # Update the screen to show the column of asterisks
        for j in range(num_asterisks):
            screen[height - 1 - j][i * width // len(data)] = '*'

    # Print the ASCII art
    for row in screen:
        print(''.join(row))



In [144]:
import os
import subprocess
import numpy as np

def check_generate_fft(N):
    filename = f"unrolled_fft_{N}.c"
    if not os.path.exists(filename):
        # Generate FFT code using Python function
        generate_fft_code_python(N, filename)
    return filename

def compile_fft(N):
    """Compile the FFT code."""
    # Compile the generated C code
    subprocess.run(["gcc", f"unrolled_fft_{N}.c", "-o", f"fft_{N}", "-lm"])

def run_fft(N, data):
    # Prepare input data for the FFT
    input_data_str = "\n".join(map(str, data)) + "\n"

    # Execute the FFT
    result = subprocess.run([f"./fft_{N}"], input=input_data_str, stdout=subprocess.PIPE, text=True)

    def extract_complex_numbers(output):
        # Split the output into individual lines
        lines = output.splitlines()

        # Extract complex numbers from each line
        complex_numbers = []
        for line in lines:
            # Split the line at the '+' sign and remove the trailing 'i'
            real_part, imag_part = line.split('+')

            # Convert the parts to float and then to a complex number
            complex_num = complex(float(real_part.strip()), float(imag_part.strip()[:-1]))

            complex_numbers.append(complex_num)

        return complex_numbers

    fft_output = extract_complex_numbers(result.stdout)

    return np.array(fft_output)



def main():
    N = 1024  # or other desired values
    waveform = top_hat(N)

    print("Input:")
    ascii_art2(np.real(waveform))

    # Apply the shifty operation on the input waveform
    waveform_shifted = [waveform[i] * (-1)**i for i in range(N)]

    print("Input (Shifted):")
    ascii_art2(np.real(waveform_shifted))

    # Compute the FFT using numpy
    fft_output_np = np.fft.fft(waveform_shifted)

    # Shift the FFT output back
    fft_output_shifted_back = [fft_output_np[i] * (-1)**i for i in range(N)]
    fft_magnitude_np = np.abs(fft_output_shifted_back)

    print("\nFFT Output using numpy:")
    ascii_art2(fft_magnitude_np)

    #this section using the generated unrolled c code
    check_generate_fft(N)
    compile_fft(N)

    # Run the unrolled FFT
    fft_result = run_fft(N, waveform_shifted)

    # Shift the unrolled FFT output back
    fft_result_shifted_back = [fft_result[i] * (-1)**i for i in range(N)]

    print("\nFFT Magnitude (Unrolled):")
    ascii_art2(np.abs(fft_result_shifted_back))


main()


Input:
                                ****************************************************************                                
                                ****************************************************************                                
                                ****************************************************************                                
                                ****************************************************************                                
                                ****************************************************************                                
                                ****************************************************************                                
                                ****************************************************************                                
                                **********************************************************

In [145]:
!ls -lash

total 2.8M
4.0K drwxr-xr-x 1 root root 4.0K Sep 19 11:54 .
4.0K drwxr-xr-x 1 root root 4.0K Sep 19 07:11 ..
4.0K drwxr-xr-x 4 root root 4.0K Sep 15 13:21 .config
772K -rwxr-xr-x 1 root root 772K Sep 19 11:54 fft_1024
 80K -rwxr-xr-x 1 root root  80K Sep 19 11:29 fft_128
 20K -rwxr-xr-x 1 root root  20K Sep 19 11:33 fft_16
164K -rwxr-xr-x 1 root root 164K Sep 19 11:31 fft_256
 28K -rwxr-xr-x 1 root root  28K Sep 19 11:01 fft_32
352K -rwxr-xr-x 1 root root 352K Sep 19 11:20 fft_512
 44K -rwxr-xr-x 1 root root  44K Sep 19 11:02 fft_64
 16K -rwxr-xr-x 1 root root  16K Sep 19 11:37 fft_8
4.0K drwxr-xr-x 1 root root 4.0K Sep 15 13:22 sample_data
708K -rw-r--r-- 1 root root 705K Sep 19 11:54 unrolled_fft_1024.c
 60K -rw-r--r-- 1 root root  60K Sep 19 11:04 unrolled_fft_128.c
8.0K -rw-r--r-- 1 root root 4.5K Sep 19 11:33 unrolled_fft_16.c
140K -rw-r--r-- 1 root root 139K Sep 19 10:07 unrolled_fft_256.c
 12K -rw-r--r-- 1 root root  11K Sep 19 11:01 unrolled_fft_32.c
316K -rw-r--r-- 1 root root 

In [143]:
!cat unrolled_fft_8.c

#include <complex.h>
#include <stdio.h>

void fft_unrolled_8(double* x, complex double* X) {
    // Stage 1
    {
        double t = x[0];
        x[0] = t + x[1];
        x[1] = t - x[1];
    }
    {
        double t = x[2];
        x[2] = t + x[3];
        x[3] = t - x[3];
    }
    {
        double t = x[4];
        x[4] = t + x[5];
        x[5] = t - x[5];
    }
    {
        double t = x[6];
        x[6] = t + x[7];
        x[7] = t - x[7];
    }

    // Stage 2
    {
        double t = x[0];
        x[0] = t + x[2];
        x[2] = t - x[2];
    }
    {
        complex double t = x[3] * (0.7071067811865476 - I * -0.7071067811865475);
        x[3] = x[1] - t;
        x[1] += t;
    }
    {
        double t = x[4];
        x[4] = t + x[6];
        x[6] = t - x[6];
    }
    {
        complex double t = x[7] * (0.7071067811865476 - I * -0.7071067811865475);
        x[7] = x[5] - t;
        x[5] += t;
    }

    // Stage 3
    {
        double t = x[0];
        x[0] = t + x[4];
      

In [146]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [147]:
!mkdir "/content/drive/My Drive/FFT_Files"
!cp fft_* "/content/drive/My Drive/FFT_Files/"
!cp unrolled_fft_* "/content/drive/My Drive/FFT_Files/"
